## 案例二 电影评分

In [8]:
import pandas as pd

mfile = 'datasets/movielens/movies.dat'
ufile = 'datasets/movielens/users.dat'
rfile = 'datasets/movielens/ratings.dat'

# 读电影数据文件
mtitles = ['movie_id', 'name', 'type']
movies = pd.read_table(mfile, names=mtitles, sep='::')

# 读用户数据文件
utitles = ['user_id', 'sex', 'age', 'job', 'zip']
users =  pd.read_table(ufile, names=utitles, sep='::')

# 读评分文件
rtitles = ['user_id', 'movie_id', 'score', 'timestamp']
scores = pd.read_table(rfile, names=rtitles, sep='::')

/Users/panliu/.local/share/virtualenvs/pydata-book-oZ7iJid9/lib/python3.7/site-packages/ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':
/Users/panliu/.local/share/virtualenvs/pydata-book-oZ7iJid9/lib/python3.7/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]
/Users/panliu/.local/share/virtualenvs/pydata-book-oZ7iJid9/lib/python3.7/site-packages/ipykernel_launcher.py:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and d

In [17]:
df1 = pd.merge(pd.merge(users, scores), movies)
df2 = df1[['name', 'sex', 'age', 'score']]
'''
name        F   M
One Flew    50  90
Broken Ves  90  56
'''
'''
name        score
One Flow    F  50
            M  90
Broken Ves  F  90
            M  56
'''
def func_group(group):
    return group['score'].sum()
df3 = df2.groupby(['name', 'sex']).apply(func_group)
df4 = df3.unstack()

sex,F,M
name,,
"$1,000,000 Duck (1971)",54.0,58.0
'Night Mother (1986),122.0,114.0
'Til There Was You (1997),99.0,41.0
"'burbs, The (1989)",257.0,625.0
...And Justice for All (1979),134.0,605.0
...,...,...
"Zed & Two Noughts, A (1985)",28.0,71.0
Zero Effect (1998),228.0,901.0
Zero Kelvin (Kj�rlighetens kj�tere) (1995),NaN,7.0
